In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format='svg'

from genominterv.decorators import bootstrap
from genominterv.stats import proximity_stat, jaccard_stat

In [18]:
def parse_compartment_data(file_name):
    e1_100kb = pd.read_csv(file_name)
    e1_100kb['start'] = [i*100_000 for i in range(e1_100kb.index.size)]
    e1_100kb['end'] = e1_100kb.start + 100_000
    e1_100kb['sign'] = np.sign(e1_100kb.e1)
    e1_100kb['segment_id'] = ((e1_100kb.sign.shift() != e1_100kb.sign)).cumsum()
    
    comp = e1_100kb.groupby('segment_id', as_index=False).agg(dict(
         e1=['mean', 'sum'], 
         start='min', 
         end='max', 
         segment_id='mean', 
         sign='mean'
    ))
    comp.columns = ['_'.join(col).strip() for col in comp.columns.values]
    comp = comp.rename(
        columns={'start_min':'start',
                 'end_max':'end', 
                 'segment_id_mean':'segment_id', 
                 'sign_mean':'sign'}
    )
    comp['comp'] = ['A' if x > 0 else 'B' for x in comp.sign]
    comp = comp.reset_index()
    comp['chrom'] = 'chrX'
    
    _comp = comp.copy()
    for i in range(1, _comp.index.size-1):
        if np.isnan(_comp.loc[i-1, 'e1_mean']):
            _comp.loc[i, 'start'] = np.nan
        if np.isnan(_comp.loc[i+1, 'e1_mean']):
            _comp.loc[i, 'end'] = np.nan
    _comp = _comp.loc[~_comp.e1_mean.isnull(), :]
    _comp = _comp.reset_index()
    compartment_edges = pd.concat([_comp.start, _comp.end]).sort_values().unique()
    
    compartments = comp.loc[~comp.e1_mean.isnull()].copy()
    compartments['start'] = compartments.start.astype(int)
    compartments['end'] = compartments.end.astype(int)

    return compartments, compartment_edges

def edge_segments(compartment_edges, flank):
    compartment_edge_segm = pd.DataFrame(np.column_stack((compartment_edges, compartment_edges+flank)), columns=['start', 'end'])
    compartment_edge_segm['chrom'] = 'chrX'
    return compartment_edge_segm

chrom_sizes = {
    'chr1': 223616942, 'chr2': 196197964, 'chr5': 187317192, 'chr3': 185288947,
    'chr6': 179085566, 'chr4': 169963040, 'chr7': 169868564, 'chrX': 153388924,
    'chr8': 145679320, 'chr9': 134124166, 'chr11': 133066086, 'chr12': 130043856,
    'chr14': 128056306, 'chr15': 113283604, 'chr13': 108737130, 'chr10': 99517758,
    'chr17': 95433459, 'chr16': 79627064, 'chr20': 77137495, 'chr18': 74474043,
    'chr19': 58315233, 'chrY': 11753682,
}

@bootstrap(chrom_sizes)
def proximity_test(q, a):
    return proximity_stat(q, a)


@bootstrap(chrom_sizes)
def jaccard_test(q, a):
    return jaccard_stat(q, a)


def overlaps(df1, df2):
    """
    Establishes whether each query segment overlaps at least one 
    annotation segment. Returns a boolean array with same length 
    as df1.index.
    """
    overlapping = []
    for i, (s1, e1) in enumerate(zip(df1.start, df1.end)):
        overlaps = False
        for s2, e2 in zip(df2.start, df2.end):
            if e1 > s2 and e2 > s1:
                overlaps = True
                break
        overlapping.append(overlaps)
    return np.array(overlapping)


all_tests = pd.read_csv('../results/all_tests.csv', index_col=0)

def svedig_tabel(orig_df, index, columns, values, 
                 cmap='Reds', col_order=None, col_snames=None):

    """
    Creates a styled pivot table from a DataFrame, filtering by significance, 
    applying a log transformation, and adding visual styling.

    Parameters:
    ----------
    orig_df : pd.DataFrame
        The original DataFrame containing data to be processed.
    index : str
        Column name to use as the row index in the pivot table.
    columns : str
        Column name to use as the column headers in the pivot table.
    values : str
        Column name whose values populate the pivot table cells.
    cmap : str, optional
        Colormap to use for the background gradient styling (default is 'Reds').

    Returns:
    -------
    pd.io.formats.style.Styler
        A styled DataFrame object with the pivot table, including gradient-based
        cell coloring, transparent NaN values, formatted numbers, and table styles.
    """
    
    df = (orig_df
     .assign(log10p=np.log10(orig_df.p))
     .loc[(orig_df[values] < 0.05)]
     .pivot(index=index, columns=columns, values=values)
    )
    if col_order:
        col_bool = [x in df.columns for x in col_order]
        col_order = [x for x in col_order if x in df.columns]
        df = df[col_order]
    if col_snames:
        col_snames = [col_snames[i] for i,x in enumerate(col_bool) if x]
        df = df.rename(columns = {x:col_snames[i] for i,x in enumerate(df.columns.tolist())})
    else:
        df = df.rename(columns = {x:x.replace('_', '<br>') for x in df.columns.tolist()})
    df = (df.style
     .background_gradient(subset=df.columns, axis=None, cmap=cmap, vmin=0)
     .map(lambda x: 'color: transparent; background-color: transparent' if np.isnan(x) else '')
     .format('{:.3f}')
     .set_table_styles(
                {c: [{'selector': '', 
                      'props': [('min-width', '20px')],
                     }] for c in df.columns}, overwrite=False
     )
    )
    return df

## Benjamini-Hochberg adjusted p-values

In [3]:

# Load the CSV file
file_path = "../results/all_tests.csv"  # Replace with the path to your CSV file
df = pd.read_csv(file_path, index_col=0)

# Check if the 'p-value' column exists
if 'p' not in df.columns:
    raise ValueError("The input file must contain a column named 'p'.")

# Define a function to apply BH correction to each group
def apply_bh_correction(group):
    group = group.sort_values('p').reset_index(drop=True)  # Sort within the group
    m = len(group)  # Total number of tests in the group
    group['rank'] = range(1, m + 1)  # Rank the p-values
    group['BH_adj_p'] = (group['p'] * m) / group['rank']  # Compute adjusted p-values
    group['BH_adj_p'] = group['BH_adj_p'].clip(upper=1)  # Ensure adjusted p-values do not exceed 1
    group['BH_adj_p'] = group['BH_adj_p'][::-1].cummin()[::-1]  # Enforce monotonicity
    return group

# Apply the function to each group
df_corrected = (df.groupby('test', group_keys=False)
                .apply(apply_bh_correction))


print(f"Benjamini-Hochberg FDR correction applied.")

df_corrected.query('p < 0.05').sort_values(['test','BH_adj_p'])[['tissue', 'query', 'annot', 'pc_scale', 'test', 'p', 'BH_adj_p']]


Benjamini-Hochberg FDR correction applied.


/tmp/ipykernel_880951/273916617.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(apply_bh_correction))


,tissue,query,annot,pc_scale,test,p,BH_adj_p
0,pachytene_spermatocyte,ECH90,comp_edge_1bp,arms,jaccard,0.00534,0.020400
1,spermatogonia,ECH90,comp_edge_1bp,10Mb,jaccard,0.00611,0.020400
2,spermatogonia,ECH90,comp_edge_1bp,arms,jaccard,0.00612,0.020400
3,fibroblast,ECH90,comp_edge_1bp,arms,jaccard,0.01656,0.041400
4,fibroblast,ECH90,comp_edge_1bp,10Mb,jaccard,0.02146,0.042920
5,round_spermatid,ECH90,comp_edge_1bp,arms,jaccard,0.02769,0.046150
6,sperm,ECH90,comp_edge_1bp,10Mb,jaccard,0.04750,0.066544
0,sperm,olivehama_edge_1bp,comp_edge_1bp,10Mb,proximity,0.00128,0.038400
1,pachytene_spermatocyte,olivehama_edge_1bp,comp_edge_1bp,10Mb,proximity,0.01230,0.140925
2,sperm,olive_edge_1bp,comp_edge_1bp,10Mb,proximity,0.01614,0.140925


In [21]:
#| label: tbl-test-results-uncorrected

tissue_order = ['fibroblast', 'spermatogonia', 'pachytene_spermatocyte', 'round_spermatid', 'sperm']
tissue_sname = ['Fb', 'Spa', 'Pac', 'RS', 'Sperm']


t0 = (svedig_tabel(all_tests
                   .map(lambda x: x.replace('_edge_1bp', '') if isinstance(x, str) else x)
                   .rename(columns={'pc_scale': 'viewframe'}),
             index=['viewframe', 'test','query'],
             columns=["tissue"],
             col_order = tissue_order,
             col_snames = tissue_sname,
             values='p',
             cmap='Reds_r')
)

display(t0)

In [20]:
#| label: tbl-test-results-BH-corrected

t1 = (svedig_tabel(df_corrected
                   .map(lambda x: x.replace('_edge_1bp', '') if isinstance(x, str) else x)
                   .rename(columns={'pc_scale': 'viewframe'}),
             index=['viewframe', 'test','query'],
             columns=["tissue"],
             col_order = tissue_order,
             col_snames = tissue_sname,
             values='BH_adj_p',
             cmap='Reds_r')
)
t1